In [1]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import timeit

from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
import glob
import pandas as pd
import numpy as np
from math import sqrt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import defaultdict

In [2]:
root_dir='./20_newsgroups_Train'

# root_dir needs a trailing slash (i.e. /root/dir/)
i = 0
train = []#pd.DataFrame(columns=['folder', 'fileName'])
for filename in glob.iglob(root_dir + '**/**/*', recursive=True):
    if i > 19:
        for j, c in enumerate(filename[22:]):
            if c.isdigit():
                bs = j
                #print(bs)
                break
        #print(filename[22:])
        #print(filename[22:22+bs-1])#lable
        label = filename[22:22+bs-1]
        #print(filename[22+bs:])#filename
        filenametmp = filename[22+bs:]
        train.append([label,filenametmp])
    #print(filename)
    i+=1
    
root_dirTest='./20_newsgroups_Test'

# root_dir needs a trailing slash (i.e. /root/dir/)
i = 0
test = []#pd.DataFrame(columns=['folder', 'fileName'])
for filename in glob.iglob(root_dirTest + '**/**/*', recursive=True):
    if i > 19:
        for j, c in enumerate(filename[21:]):
            if c.isdigit():
                bs = j
                #print(bs)
                break
        #print(filename[22:])
        #print(filename[22:22+bs-1])#lable
        label = filename[21:21+bs-1]
        #print(filename[22+bs:])#filename
        filenametmp = filename[21+bs:]
        test.append([label,filenametmp])
    #print(filename)
    i+=1

In [3]:
allDocsAsStringsTrain=[]
allDocsAsStringsNoLabelsTrain = []
lastrow=""

category = -1
ytest = []
ytrain = []
for row in train:
    fileToOpen= root_dir+'/'+row[0]+'/'+row[1]
    
    file=open(fileToOpen,"r")
    tmp = file.read().lower()#text_cleaner(file.read()).lower()
    
    if (row[0]!= lastrow):
        lastrow = row[0]
        category += 1
    allDocsAsStringsTrain.append([row[0],row[1],tmp])
    ytrain.append(category)
    allDocsAsStringsNoLabelsTrain.append(tmp)
    file.close()
    
allDocsAsStringsTest=[]
allDocsAsStringsNoLabelsTest = []
lastrow=""
category = -1
for row in test:
    fileToOpen= root_dirTest+'/'+row[0]+'/'+row[1]
    if (row[0]!= lastrow):
        lastrow = row[0]
        category+=1
    
    #print(fileToOpen)
    file=open(fileToOpen,"r")
    tmp = file.read().lower()#text_cleaner(file.read()).lower()
    ytest.append(category)
    allDocsAsStringsTest.append([row[0],row[1],tmp])
    allDocsAsStringsNoLabelsTest.append(tmp)
    file.close()

In [4]:
corpus = []
i=0
#ytrain = []
for doc in allDocsAsStringsNoLabelsTrain:
    corpus.append(''.join((element for element in doc if not (element.isdigit() or element =='_'))))
    #if i%10 == 0:
    #    print(doc)
    #ytrain.append(allDocsAsStringsTrain[i][3])
    i=+1
    
#ytest = []  
corpusTest = []
i=0
for doc in allDocsAsStringsNoLabelsTest:
    corpusTest.append(''.join((element for element in doc if not (element.isdigit() or element =='_'))))
    #ytest.append(allDocsAsStringsTest[i][3])

    i=+1
ytrain=np.array(ytrain)
ytest =np.array(ytest)


In [6]:
wordLimit=10000

In [7]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english', lowercase=True,max_features=wordLimit)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

text_clf.fit(corpus, ytrain)



Pipeline(steps=[('vect',
                 CountVectorizer(max_features=10000, stop_words='english')),
                ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])

In [8]:
start = timeit.default_timer()
predicted = text_clf.predict(corpusTest)
stop = timeit.default_timer()

In [9]:
elapsed = stop-start
print(elapsed)

2.0315942999999947


In [10]:
predicted

array([ 0,  0,  0, ..., 19, 19,  0])

In [11]:
cases = len(predicted)

index = 0
right = 0
wrong = 0
recalls = np.zeros([category+1,2])
for i in predicted:
    j=-1
    
    pred = i
    
    if pred == int(ytest[index]):
        recalls[pred][0]+=1
        right +=1
    else:
        recalls[pred][1]+=1
        wrong+=1
    index+=1

In [12]:
rec = 0
for row in recalls:
    rec+=(row[0]/(row[0]+row[1])) 
rec = rec/(category+1)

In [13]:
print(wrong)
print(right/(right+wrong))

681
0.943240540090015


In [14]:
rec

0.9442626392867256